<a href="https://colab.research.google.com/github/Niwanka/IRWA/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import heapq

In [5]:
df=pd.read_csv("https://raw.githubusercontent.com/Niwanka/IRWA/main/small_dataset.csv?token=GHSAT0AAAAAACXE3IX4R3JXUVT6DGYAXOAQZXMKZRA")

In [6]:
df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    1000 non-null   int64  
 1   Name                        1000 non-null   object 
 2   AuthorId                    1000 non-null   int64  
 3   AuthorName                  1000 non-null   object 
 4   CookTime                    625 non-null    object 
 5   PrepTime                    1000 non-null   object 
 6   TotalTime                   1000 non-null   object 
 7   DatePublished               1000 non-null   object 
 8   Description                 1000 non-null   object 
 9   Images                      1000 non-null   object 
 10  RecipeCategory              1000 non-null   object 
 11  Keywords                    990 non-null    object 
 12  RecipeIngredientQuantities  999 non-null    object 
 13  RecipeIngredientParts       1000 n

In [8]:
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [9]:
df['Name'] = df['Name'].apply(clean_text)
df['Description'] = df['Description'].apply(clean_text)
df['Keywords'] = df['Keywords'].apply(clean_text)

In [10]:
df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,lowfat berry blue frozen dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,make and share this lowfat berry blue frozen d...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,make and share this biryani recipe from foodcom,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,best lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,this is from one of my first good house keepin...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,carinas tofuvegetable kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,this dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,cabbage soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,make and share this cabbage soup recipe from f...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [11]:
df['text'] = df['Name'] + ' ' + df['Description'] + ' ' + df['Keywords']

In [12]:
df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,text
0,38,lowfat berry blue frozen dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,make and share this lowfat berry blue frozen d...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",lowfat berry blue frozen dessert make and shar...
1,39,biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,make and share this biryani recipe from foodcom,"c(""https://img.sndimg.com/food/image/upload/w_...",...,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and...",biryani make and share this biryani recipe fro...
2,40,best lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,this is from one of my first good house keepin...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ...",best lemonade this is from one of my first goo...
3,41,carinas tofuvegetable kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,this dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",carinas tofuvegetable kebabs this dish is best...
4,42,cabbage soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,make and share this cabbage soup recipe from f...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil...",cabbage soup make and share this cabbage soup ...


In [13]:
df.dropna(subset=['text'], inplace=True)

In [14]:
# Initialize TF-IDF Vectorizer and transform dataset
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Get idf values for pruning low-idf terms
idf_values = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))


In [15]:
# Function to prune low-idf terms from a query
def prune_query(query, threshold=1.5):
    terms = query.split()
    # Retain only terms with high idf values
    pruned_terms = [term for term in terms if idf_values.get(term, 0) >= threshold]
    return ' '.join(pruned_terms)

In [16]:
# Function to compute top-K cosine similarities using a heap
def efficient_top_k_cosine(query_vec, top_k=5):
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Use heapq to find the top K cosine scores
    top_indices = heapq.nlargest(top_k, range(len(cosine_similarities)), key=cosine_similarities.__getitem__)
    return top_indices

In [17]:
def search_recipes(query, top_k=5):
    # Clean and prune the query using high-idf terms
    query = clean_text(query)
    query = prune_query(query)

    if not query:
        return pd.DataFrame(columns=['Name', 'Description', 'Keywords'])  # Return empty if the query has no high-idf terms

    # Convert the query into a TF-IDF vector
    query_vec = vectorizer.transform([query])

    # Efficient top-K cosine ranking using heap
    top_indices = efficient_top_k_cosine(query_vec, top_k)

    # Return the most similar recipes
    return df.iloc[top_indices][['Name', 'Description', 'Keywords']].reset_index(drop=True)


In [18]:
query = "breakfast"
results = search_recipes(query)
print(results)

                       Name  \
0        breakfast burritos   
1  baked breakfast potatoes   
2        breakfast pizza ii   
3           apricot muffins   
4           english muffins   

                                         Description  \
0  make and share this breakfast burritos recipe ...   
1  make and share this baked breakfast potatoes r...   
2  make and share this breakfast pizza ii recipe ...   
3  make and share this apricot muffins recipe fro...   
4  make and share this english muffins recipe fro...   

                                            Keywords  
0                              cmexican 60 mins easy  
1           cpotato vegetable weeknight oven 4 hours  
2                cpork cheese meat 30 mins oven easy  
3                     cbreads breakfast 60 mins oven  
4  cbreads breakfast european healthy weeknight s...  
